# Introduction
This 

In [12]:
import numpy as np

In [37]:
class Network:
    
    def __init__(self, architecture, activation= 'sigmoid', learning_rate= 0.001):
        self.architecture= architecture
        #self.activation= activation
        self.lr= learning_rate
        self.parameters=[]
        self.buffered_op={}
        #self.prime_funcs=[]
        
        if isinstance(activation, str):
            if activation.lower().strip() in ['relu', 'sigmoid']:
                if activation.lower().strip()== 'relu':
                    self.activation= [self.relu for x in range(len(self.architecture) - 1)]
                    self.prime_funcs = [self.relu_prime for x in range(len(self.architecture) - 1)]
                else:
                    self.activation= [self.sigmoid for x in range(len(self.architecture) - 1)]
                    self.prime_funcs = [self.sigmoid_prime for x in range(len(self.architecture) - 1)]
                    
            else:
                raise ValueError("activation Value should be either relu or sigmoid")
                    
        
        for i in range(len(architecture) -1):
            layer_parameters= self.create_weights_and_biases(architecture[i], architecture[i + 1])
            self.parameters.append(layer_parameters)
        
        
    def create_weights_and_biases(self, in_nodes, out_nodes):
        std= np.sqrt(2.0/(in_nodes + out_nodes))
        w= np.random.normal(loc= 0, scale= std, size= (in_nodes, out_nodes))
        b= np.zeros((1, out_nodes))
        return {'weight':w , 'bias':b}
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return 0 if x<0 else x
    
    def sigmoid_prime(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    def relu_prime(self, x):
        return 1 if x>0 else 0

    def forward_pass(self, x):
        if x.ndim == 1:
            x= x.reshape((1, -1))
        self.buffered_op['a0']= x
        for i, parameter in enumerate(self.parameters):
            w= parameter['weight']
            b= parameter['bias']
            actv_func= self.activation[i]
            
            h= np.matmul(x, w) + b
            a= actv_func(h)
            self.buffered_op['h' + str(i+1)]= h
            self.buffered_op['a' + str(i+1)]= a
            x= a
        return a
    
    def loss(self, y_cap, y):
        return (y - y_cap) ** 2 #MSE
    
    def loss_prime(self, y_cap, y):
        return -2 * (y - y_cap)
    
    def backpropagation(self,y_cap, y):
        #last_layer= True
        layer_error= self.loss_prime( y_cap, y)
        gradients= []
        batch_size= y_cap.shape[0]
        
        
        for layer_idx in range(len(self.architecture) - 1, 0 , -1):
            prime_func= self.prime_funcs[layer_idx - 1]
            h_cur= self.buffered_op['h'+str(layer_idx)]
            a_prev= self.buffered_op['a'+str(layer_idx - 1)]
            
            
            error_term= layer_error * prime_func(h_cur)
            del_w= np.matmul(error_term.T , a_prev)
            del_b= error_term.sum(axis= 0)
            gradients.append((del_w, del_b))
            
            #updating layer_error term for next iteration
            layer_error= np.matmul(error_term , self.parameters[layer_idx -1]['weight'].T)
            
        gradients.reverse()
        
        #updating the weights:
        for i in range(len(self.parameters)):
            self.parameters[i]['weight'] -= (self.lr / batch_size) * gradients[i][0].T
            self.parameters[i]['bias'] -= (self.lr / batch_size) * gradients[i][1]
            
    def train(self, train_x, train_y, epochs= 5, learning_rate= 0.01, batch_size= 1):
        no_of_records= train_x.shape[0]
        self.lr= learning_rate
    
        for epoch in range(epochs):
            epoch_error= 0
            for n in range(batch_size, no_of_records, batch_size):
                x_batch= train_x[n- batch_size: n]
                y_batch= train_y[n- batch_size: n]
        
                y_cap= self.forward_pass(x_batch)
                batch_error= self.loss(y_cap, y_batch)
                #print(batch_error.shape)
                epoch_error+= batch_error.sum()
                self.backpropagation(y_cap, y_batch)
        
            print("Epoch {} - Training Loss - {}\n".format(epoch+1, epoch_error))
            
            

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
data= load_breast_cancer()
x= data.data
y= data.target
cols= data.target_names
print(type(x))
print(x.shape)

sklearn.utils.Bunch

In [9]:
scaler= StandardScaler()
scaler.fit(x)
scaled_x= scaler.transform(x)

In [10]:
y[:5]

array([0, 0, 0, 0, 0])

In [17]:
y= y.reshape((-1,1))
x_train, x_test, y_train, y_test= train_test_split(scaled_x, y, test_size= 0.2, 
                                                   shuffle= True, random_state= 711)
print(x_train.shape)

(455, 30)


In [38]:
neu_net= Network([30, 10, 5, 1])
neu_net.train( x_train, y_train, epochs= 25, learning_rate= 0.1, batch_size= 8)

Epoch 1 - Training Loss - 102.66467232428076

Epoch 2 - Training Loss - 99.95026271754116

Epoch 3 - Training Loss - 97.72598637856396

Epoch 4 - Training Loss - 94.80796355620676

Epoch 5 - Training Loss - 90.92556572092928

Epoch 6 - Training Loss - 85.8607762992137

Epoch 7 - Training Loss - 79.4910720040271

Epoch 8 - Training Loss - 71.91720291881406

Epoch 9 - Training Loss - 63.56952375091343

Epoch 10 - Training Loss - 55.154046178245856

Epoch 11 - Training Loss - 47.38928911171898

Epoch 12 - Training Loss - 40.72677981003927

Epoch 13 - Training Loss - 35.27802826260151

Epoch 14 - Training Loss - 30.926439963211653

Epoch 15 - Training Loss - 27.469264711094166

Epoch 16 - Training Loss - 24.7046041637634

Epoch 17 - Training Loss - 22.464695638282887

Epoch 18 - Training Loss - 20.62135010259302

Epoch 19 - Training Loss - 19.08241642887373

Epoch 20 - Training Loss - 17.786152646219524

Epoch 21 - Training Loss - 16.69105515184859

Epoch 22 - Training Loss - 15.7627818618

In [41]:
def acuracy_check_binary_classifier(predicted, actual):
    y_test_cap= np.zeros_like(predicted)
    y_test_cap[predicted >= 0.5] = 1
    correct_pred_count= (y_test_cap== actual).sum()
    return 100 * correct_pred_count / len(actual)

In [42]:
pred= neu_net.forward_pass(x_test)
print(acuracy_check_binary_classifier(pred, y_test))

96.49122807017544
